<a href="https://colab.research.google.com/github/hawc2/Webscraping-Project-Gutenberg/blob/master/Gutenberg_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gutenberg API

Two options

1) https://github.com/ageitgey/Gutenberg

I think I've got 1) working! Still has some things to fix thouhg


2) Backup Option: https://github.com/c-w/gutenberg-http/



# Installing Libraries

In [ ]:
!git clone https://github.com/c-w/Gutenberg.git
%cd Gutenberg

In [ ]:
!sudo apt-get install python-virtualenv

In [ ]:
!virtualenv --no-site-packages virtualenv
!source virtualenv/bin/activate

In [ ]:
YES_I_HAVE_THE_RIGHT_TO_USE_THIS_BERKELEY_DB_VERSION=1
!sudo apt-get install -y python3-bsddb3 
!sudo apt-get install libdb++-dev
!export BERKELEYDB_DIR=/usr
!pip install .

In [ ]:
!pip3 install -r requirements-dev.pip
!pip3 install .

!pip3 install nose
!nose2

# Test Query

Without setting Gutenberg mirror, I got this message:
UnknownDownloadUriException: Could not reach Gutenberg mirror "http://aleph.gutenberg.org". Try setting a different mirror (https://www.gutenberg.org/MIRRORS.ALL) for --mirror flag or GUTENBERG_MIRROR environment variable.

Ideally set the alternate mirror permanently so it doesn't need to be a paremeter in query request

This works: https://gutenberg.pglaf.org/

In [ ]:
#THIS WORKS!
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers

text = strip_headers(load_etext(2701, mirror = "https://gutenberg.pglaf.org/")).strip()
print(text)  # prints 'MOBY DICK; OR THE WHALE\n\nBy Herman Melville ...'

In [ ]:
#translate from unix to python
python -m gutenberg.acquire.text 2701 moby-raw.txt
python -m gutenberg.cleanup.strip_headers moby-raw.txt moby-clean.txt

In [ ]:
from gutenberg.query import get_etexts
from gutenberg.query import get_metadata

print(get_metadata('title', 2701))  # prints frozenset([u'Moby Dick; Or, The Whale'])
print(get_metadata('author', 2701)) # prints frozenset([u'Melville, Hermann'])

print(get_etexts('title', 'Moby Dick; Or, The Whale'))  # prints frozenset([2701, ...])
print(get_etexts('author', 'Melville, Hermann'))        # prints frozenset([2701, ...])

# Cache

I'm confused when we need to do this. The line of code immediately above this comment gets an eror: "The cache is invalid or not created"

The developer claims: "# Before you use one of the gutenberg.query functions you must populate the local metadata cache. This one-off process will take quite a while to complete (18 hours on my machine) but once it is done, any subsequent calls to get_etexts or get_metadata will be very fast. If you fail to populate the cache, the calls will raise an exception."

In [ ]:
#To populate the cache:

from gutenberg.acquire import get_metadata_cache
cache = get_metadata_cache()
cache.populate()

In [ ]:
#If you need more fine-grained control over the cache (e.g. where it’s stored or which backend is used), you can use the set_metadata_cache function to switch out the backend of the cache before you populate it. For example, to use the Sqlite cache backend instead of the default Sleepycat backend and store the cache at a custom location, you’d do the following:

from gutenberg.acquire import set_metadata_cache
from gutenberg.acquire.metadata import SqliteMetadataCache

cache = SqliteMetadataCache('/my/custom/location/cache.sqlite')
cache.populate()
set_metadata_cache(cache)